In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
# Set URL and browser
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/ignacioguerra/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


### Visit the NASA Mars News Site

In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)

# Search for elements with a specific combination of tag (div) and 
# attribute (list_text). Add an optional delay of 1 second in case
# the page is slow to load.
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# Set up the HTML parser:
html = browser.html

# Convert the browser html to a soup object and then quit the browser
news_soup = soup(html, 'html.parser')

# Use select_one to only select the first item, as we only want
# the most recent Nasa article on the page.
# Use div.list_text to search for all 'div' tag with a class 'list_text'
slide_elem = news_soup.select_one('div.list_text')

In [5]:
# After opening the web page and using Inspector to explore 
# the most recent article, we identified that the title is in 
# class 'content_title'.

# look inside the slide_elem object for the specific title
slide_elem.find('div', class_='content_title')

<div class="content_title">InSight's 'Mole' Team Peers into the Pit</div>

In [6]:
# Clean the title of other html code. 
# Use the parent element to find the first `a` tag and 
# save it as `news_title`.
# This line is similar to the code above, but adds the 
# method "get_text()" to extract only the text from the html.
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"InSight's 'Mole' Team Peers into the Pit"

In [7]:
# Now get the teaser body. After using Inspector on that element of
# the web page, we identied it is associated with class "article_teaser_body."
# Repurpose the above code to get it.
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Efforts to save the heat probe continue.'

### JPL Space Images Featured Image

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
### WHY USE A [1] IF THERE ARE PREVIOUS 'button' INSTANCES IN THE INSPECTOR??
### THE PREVIOUS BUTTON IS FOR THE SEARCH BAR, IF YOUR USE [0], THAT ONE
### WILL BE SELECTED!!!

# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')
img_soup

<html class="fancybox-margin fancybox-lock"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:0px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-to

In [11]:
# find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [12]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

## MODULE 10.3.5 - Scrape Mars Data: Mars Facts

In [13]:
#########################################################
# For scraping the data, we will use Pandas .read_html()
# method to read whole tables instead of item by item
#########################################################

### Mars Facts

In [14]:
# Create a new DataFrame from the HTML table using
# the Pandas read_html method. By specifying an index 
# of [0], we're telling Pandas to pull only the first table 
# it encounters (or the first item in the list). Then, it 
# turns the table into a DataFrame.
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [15]:
# Assign columns to the new DataFrame for additional clarity.
df.columns=['Description', 'Mars', 'Earth']

# Set the 'Description' column into the DataFrame's index. 
# inplace=True means that the updated index will remain in 
# place, without having to reassign the DataFrame to a new 
# variable.
df.set_index('Description', inplace=True)

# Inspect the dataframe.
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [16]:
# Pandas also has a way of converting a dataframe back to 
# an HTML table by using the to_html() method.
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

## Module 10 Challenge: DELIVERABLE 1 - Scrape High-Resolution Mars' Hemisphere Images and Titles

### Hemisphere

In [17]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [18]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

In [19]:
# 3. Write code to retrieve the image urls and titles for each hemisphere.

html = browser.html
img_soup = soup(html, 'html.parser')
img_soup

<html lang="en"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
<meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="css/main.css" media="screen" rel="stylesheet"/>
<link href="css/print.css" media="print" rel="stylesheet"/>
<link href="#" rel="icon" type="image/x-ico"/>
</head>
<body id="results">
<header>
<a href="#" style="float:right;margin-top:10px;" target="_blank">
<img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_2x.png"/>
</a>
<a href="#" style="float:

In [20]:
# Find the relevant content section in the html block
relevant_section = img_soup.find('div', class_='collapsible results')

In [21]:
# Use a for loop to: 
# CHANGE CODE TO CLICK LINKS !!!!!
# a) click on each hemisphere link, 
# b) navigate to the full-resolution image page, 
# c) retrieve the full-resolution image URL string and title for the hemisphere image,
# d) use browser.back() to navigate back to the beginning to get the next hemisphere image.

for titles in relevant_section:
    
    try:
        # Get the hemisphere image titles
        image_title = titles.find('h3').text
        print(image_title)
        
        # Get the thumbnail url
        thumbnail_url = titles.find('img', class_='thumb').get('src')
        print(thumbnail_url)
        
        img_link = titles.a['href']
        print(img_link)
        
        #complete the url address
        img_link_url = url + img_link
        print(img_link_url)
        
        #visit the image url
        browser.visit(img_link_url)
        html_2 = browser.html
        img_soup_2 = soup(html_2, 'html.parser')
        
        # exctarct the jpg image url
        image_url= url + img_soup_2.find('img', class_='wide-image').get('src')
        print(image_url)
        
        #Add image title and url to the list
        hemisphere_image_urls.append({'img_url':image_url,'title':image_title})
        
    except AttributeError:
        None

Cerberus Hemisphere Enhanced
images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png
cerberus.html
https://marshemispheres.com/cerberus.html
https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
Schiaparelli Hemisphere Enhanced
images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png
schiaparelli.html
https://marshemispheres.com/schiaparelli.html
https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
Syrtis Major Hemisphere Enhanced
images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png
syrtis.html
https://marshemispheres.com/syrtis.html
https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
Valles Marineris Hemisphere Enhanced
images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png
valles.html
https://marshemispheres.com/valles.html
https://marshemispheres.com/images/

In [22]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [23]:
# End the browsing session.
browser.quit()